In [1]:
import os
import pickle
import faiss
import random
import numpy as np
import pandas as pd

from fetch_pt_report import *
from llm_prompt import *
from fetch_xgboost_report import *

In [2]:
# """Load data from results folder"""
# # Define the path to the results folder
# results_folder = 'results'
# 
# # Open the pkl file and load its content
# with open("./results/rag_dict.pkl", 'rb') as file:
#     all_dict = pickle.load(file)
# 
# all_fids = list(all_dict['demographic_data'].keys())
# tst_perc = .05
# max_tr_num = tst_perc * len(all_fids)
# random_idxs = random.sample(range(len(all_fids)), int(max_tr_num))
# tst_fids = [fid for itr, fid in enumerate(all_fids) if itr in random_idxs]
# 
# tr_dict = {}
# tst_dict = {}
# 
# for form_nm, form_dict in all_dict.items():
#     tr_dict[form_nm] = {}
#     tst_dict[form_nm] = {}
#     for fid, instances in form_dict.items():
#         if fid in tst_fids:
#             tst_dict[form_nm][fid] = {}
#         else:
#             tr_dict[form_nm][fid] = {}
#         for inst, features in instances.items():
#             if fid in tst_fids:
#                 tst_dict[form_nm][fid][inst] = {}
#             else:
#                 tr_dict[form_nm][fid][inst] = {}
#             for feat_type, feat_vals in features.items():
#                 if fid in tst_fids:
#                     tst_dict[form_nm][fid][inst][feat_type] = {}
#                 else:
#                     tr_dict[form_nm][fid][inst][feat_type] = {}
#                 for col_name, val in feat_vals.items():
#                     if fid in tst_fids:
#                         tst_dict[form_nm][fid][inst][feat_type][col_name] = val
#                     else:
#                         tr_dict[form_nm][fid][inst][feat_type][col_name] = val
                    

In [3]:
def pad_nulls(dict_oi):
    max_lens = {key: 0 for key in dict_oi.keys()}
    for form_nm in dict_oi.keys():
        form_dict = dict_oi[form_nm]
        for fid, instances in form_dict.items():
            for inst, features in instances.items():
                if 'text_embeddings' in features.keys():
                    embed_lists = instances[inst]['text_embeddings']
                    for key in embed_lists.keys():
                        embed_list_curr = embed_lists[key]
                        if embed_list_curr is None:
                            continue
                        if max_lens[form_nm] < len(embed_list_curr):
                            max_lens[form_nm] = len(embed_list_curr)
    
    for form_nm in dict_oi.keys():
        form_dict = dict_oi[form_nm]
        for fid, instances in form_dict.items():
            for inst, features in instances.items():
                if 'text_embeddings' in features.keys():
                    embed_lists = instances[inst]['text_embeddings']
                    for key in embed_lists.keys():
                        embed_list_curr = embed_lists[key]
                        if embed_list_curr is None:
                            dict_oi[form_nm][fid][inst]['text_embeddings'][key] = [np.nan for np.nan in range(max_lens[form_nm])]
                        elif len(embed_list_curr) < max_lens[form_nm]:
                            dict_oi[form_nm][fid][inst]['text_embeddings'][key] = [np.nan for np.nan in range(max_lens[form_nm])]
                            
    return dict_oi

In [4]:
# tr_dict = pad_nulls(tr_dict)
# tst_dict = pad_nulls(tst_dict)

In [5]:
# Path to save/load the normalized numeric data dictionary
if not os.path.exists('./results'):
    os.mkdir('./results')

tr_path = "tr_dict.pkl"
tst_path = 'tst_dict.pkl'
# 
# # Save the dictionary
# with open('./results/' + tr_path, "wb") as f:
#     pickle.dump(tr_dict, f)
#     
# # Save the dictionary
# with open('./results/' + tst_path, "wb") as f:
#     pickle.dump(tst_dict, f)

results_folder = 'results'

# Open the pkl file and load its content
with open("./results/" + tr_path, 'rb') as file:
    tr_dict = pickle.load(file)
    
with open("./results/" + tst_path, 'rb') as file:
    tst_dict = pickle.load(file)

In [6]:
# def fetch_index(data):
    # Suppose we have all data instances stored in `data`
# Flatten all instances
def build_faiss(tr_dict):
    faiss_idxs = {key: '' for key in tr_dict.keys()}
    impute_dict = {form_nm: [] for form_nm in tr_dict.keys()}
    faiss_mappings = {key: [] for key in tr_dict.keys()}  # To store mappings
    for form_nm in tr_dict.keys():
        form_vectors = []
        mapping = []
        form_dict = tr_dict[form_nm]
        for fid, instances in form_dict.items():
            for inst, features in instances.items():
                # Concatenate data
                
                # if form_nm != 'cmr':
                norm_vector = [val for val in features['normalized_data'].values()]
                cat_vector = [val for val in features['categorical_data'].values()]  # Adjust as per encoding
                # meta_vector = [val for val in features['metadata'].values()]
                
                text_vector = []
                if 'text_embeddings' in features.keys():
                    # Concatenate text embeddings
                    for emb in features['text_embeddings'].values():
                        text_vector.extend(emb)
                
                final_vector = np.array(norm_vector + cat_vector + text_vector, dtype=np.float32)
                    
                # else:
                #     norm_vector = [val for val in features['normalized_data'].values()]
                #     final_vector = np.array(norm_vector, dtype=np.float32)
                form_vectors.append(final_vector)
                
                # Append the (fid, inst) to the mapping
                mapping.append((fid, inst))
    
        # Stack all vectors into a 2D array
        form_vectors = np.vstack(form_vectors)
        form_vector_df = pd.DataFrame(form_vectors)
        form_vectors = form_vector_df.fillna(form_vector_df.median()).to_numpy()
        impute_dict[form_nm] = form_vector_df.median(axis=0)
        print(f"Data shape for for form: {form_nm} ", form_vectors.shape)  # Should be (num_instances, d_total)
        
        # 3. Feed Data into FAISS
        
        # Set the dimension based on d_total
        d_total = form_vectors.shape[1]
        
        # Initialize FAISS index (flat index here, consider other types based on your data)
        index = faiss.IndexFlatIP(d_total)  # Using cosine similarity
        
        # Add vectors to the index
        index.add(form_vectors)
        faiss_idxs[form_nm] = index
        faiss_mappings[form_nm] = mapping
        print("FAISS index contains:", index.ntotal, "items")
    return faiss_idxs, impute_dict, faiss_mappings

faiss_idxs, impute_dict, faiss_mappings = build_faiss(tr_dict)

Data shape for for form: pt_clinical_surgical_history  (3557, 26183)
FAISS index contains: 3557 items
Data shape for for form: cmr  (3557, 7012)
FAISS index contains: 3557 items
Data shape for for form: catheterization  (2851, 1558)
FAISS index contains: 2851 items
Data shape for for form: echocardiogram  (4265, 3858)
FAISS index contains: 4265 items
Data shape for for form: exercise_stress_test  (1648, 786)
FAISS index contains: 1648 items
Data shape for for form: pt_status_event_update  (3557, 33125)
FAISS index contains: 3557 items
Data shape for for form: demographic_data  (3557, 3)
FAISS index contains: 3557 items


In [7]:
def test_faiss_index(faiss_indices, tst_dict, impute_dict, faiss_mappings, top_k=5):
    results = {}

    for form_nm, form_data in tst_dict.items():
        if form_nm not in faiss_indices:
            print(f"No FAISS index found for form {form_nm}")
            continue

        # Retrieve the specific FAISS index and mapping for the current form
        faiss_index = faiss_indices[form_nm]
        impute_data = impute_dict[form_nm]
        mapping = faiss_mappings[form_nm]  # Get mapping for current form
        form_results = {}

        # Construct query vectors for each item in tst_dict
        for fid_n, instances in form_data.items():
            instance_results = {}

            for inst_i, inst_data in instances.items():
                # Build the query vector
                norm_vector = [val for val in inst_data['normalized_data'].values()]
                cat_vector = [val for val in inst_data['categorical_data'].values()]

                text_vector = []
                if 'text_embeddings' in inst_data:
                    for emb in inst_data['text_embeddings'].values():
                        text_vector.extend(emb)

                # Combine all parts into a single vector
                query_vector = np.array(norm_vector + cat_vector + text_vector, dtype=np.float32).reshape(1, -1)
                query_vector = pd.DataFrame(query_vector).fillna(impute_data).to_numpy()

                # Ensure the query vector dimension matches the FAISS index dimension
                if query_vector.shape[1] != faiss_index.d:
                    print(f"Dimension mismatch: Query vector is {query_vector.shape[1]} but FAISS index expects {faiss_index.d}")
                    continue

                # Perform the search
                distances, indices = faiss_index.search(query_vector, top_k)

                # Map FAISS indices back to original force_id and instance
                neighbor_results = []
                for idx, dist in zip(indices[0], distances[0]):
                    if idx < len(mapping):  # Ensure the index is within bounds
                        fid, inst = mapping[idx]
                        neighbor_results.append({
                            'force_id': fid,
                            'instance': inst,
                            'index': idx,
                            'distance': dist
                        })

                # Store the results for the current instance
                instance_results[inst_i] = neighbor_results

            # Store each instance's result under the fid_n level
            form_results[fid_n] = instance_results

        # Store results per form type
        results[form_nm] = form_results

    return results


In [8]:
# # getting closest pts based on faiss results for test pts
top_k_matches = 25
# faiss_results = test_faiss_index(faiss_idxs, tst_dict, impute_dict, faiss_mappings, top_k=top_k_matches)
# # 
# # # Path to save the normalized numeric data dictionary
# # if not os.path.exists('./results'):
# #     os.mkdir('./results')
# # 
save_path = "faiss_index25.pkl"
# # 
# # Save the dictionary
# with open('./results/' + save_path, "wb") as f:
#     pickle.dump(faiss_results, f)

# Open the pkl file and load its content
with open("./results/" + save_path, 'rb') as file:
    faiss_results = pickle.load(file)

In [9]:
# # Check results forms and fids
for form_nm, form_results in faiss_results.items():
    # form_curr = tst_dict[form_nm]
    if form_nm == 'cmr' or form_nm == 'pt_clinical_surgical_history' or form_nm == 'pt_status_event_update':
        for fid_n, instances in form_results.items():
            for inst_i, neighbors in instances.items():
                print(f"Test instance {inst_i} from {fid_n}:")
                for neighbor in neighbors:
                    print(f"  Neighbor: {neighbor['force_id']} (instance: {neighbor['instance']}) with distance {neighbor['distance']}")


Test instance 0 from ARK-BROKAZ-1:
  Neighbor: ARK-BANALE-1 (instance: 0) with distance 5123909120.0
  Neighbor: MOT-HOLCAT-1 (instance: 0) with distance 5123908608.0
  Neighbor: NAT-CLUZOP-1 (instance: 0) with distance 5123900416.0
  Neighbor: NAT-EEKERD-1 (instance: 0) with distance 5123899904.0
  Neighbor: MTS-MAHDAV-1 (instance: 0) with distance 5123899904.0
  Neighbor: CHP-FAGLAS-1 (instance: 0) with distance 5123898880.0
  Neighbor: CHP-DADMEG-1 (instance: 0) with distance 5123898880.0
  Neighbor: YAL-CHACIN-1 (instance: 0) with distance 5123890176.0
  Neighbor: CHP-BUCFRA-1 (instance: 0) with distance 5123890176.0
  Neighbor: PIT-RODYAM-1 (instance: 0) with distance 5123880960.0
  Neighbor: CHP-BELLEN-1 (instance: 0) with distance 5123880960.0
  Neighbor: OUH-HOLDAN-1 (instance: 0) with distance 5123879936.0
  Neighbor: CHP-CONHAN-1 (instance: 0) with distance 5123878912.0
  Neighbor: NAT-WAIFUA-1 (instance: 0) with distance 5123877888.0
  Neighbor: TSK-000133-1 (instance: 0) wi

In [10]:
"""Asking Chat-GPT for responses for test patients using 2 agents (one to summarize report, one to utilize report and patient data )"""

'Asking Chat-GPT for responses for test patients using 2 agents (one to summarize report, one to utilize report and patient data )'

In [11]:
"""Load data from results folder"""
import pickle
# Define the path to the results folder
results_folder = 'results'

# Open the report formatted data pkl file and load its content
with open("./results/report_data.pkl", 'rb') as file:
    report_data_dict = pickle.load(file)
    
# Open the report formatted data pkl file and load its content
with open("./results/xgboost_data.pkl", 'rb') as file:
    xgboost_data_dict = pickle.load(file)

In [12]:
"""
Goal: for each patient:
1) aggregate top k (3) closest patient forms for given forms: cmr, pt_clinical_surgical_history (pthx).
2) aggregate reports for all top k closest cases for cmr/pthx 
2b) aggregate reports patient of interest for cmr/pthx
3) predict set of outcomes outcomes in pt_status_event_update (psu)
"""

'\nGoal: for each patient:\n1) aggregate top k (3) closest patient forms for given forms: cmr, pt_clinical_surgical_history (pthx).\n2) aggregate reports for all top k closest cases for cmr/pthx \n2b) aggregate reports patient of interest for cmr/pthx\n3) predict set of outcomes outcomes in pt_status_event_update (psu)\n'

In [13]:
tr_fids = list(tr_dict['demographic_data'].keys())
tst_fids = list(tst_dict['demographic_data'].keys())

unlabel_psu_grps = pd.read_csv('Pt Status Update Table Associated Variables.csv')
unlabel_psu_grps = [[val for val in vals if str(val) != 'nan'] for vals in unlabel_psu_grps.values]
grp_leads_to_keep_unlab = ['hx_seizures',
                           'hx_ep_pacemaker',
                           'hx_ascites',
                           'status_death',
                           'hx_tx_received'
                           ]

label_psu_grps = pd.read_csv('Labeled Pt Status Update Table Associated Variables.csv')
label_psu_grps = [[val for val in vals if str(val) != 'nan'] for vals in label_psu_grps.values]
grp_leads_to_keep = ['Does the patient have a diagnosis of seizures requiring antiepileptic medications?',
                     'Does the patient have a history of pacemaker placement?',
                     'Does the patient have a history of ascites (> trace)?',
                     'Has the patient died?',
                     'Did the patient receive a heart transplant?'
                     ]

In [14]:
"""Fetching xgboost predictions report for each outcome we hope to predict"""
from fetch_xgboost_report import *
xgboost_data_df = pd.DataFrame(xgboost_data_dict)

# separating out xgboost data df into tr/tst
xgboost_data_df_tr = xgboost_data_df[xgboost_data_df['force_id'].isin(tr_fids)]
xgboost_data_df_tst = xgboost_data_df[xgboost_data_df['force_id'].isin(tst_fids)]

xgboost_trained_models = train_xgboost_models(xgboost_data_df_tr, unlabel_psu_grps, grp_leads_to_keep_unlab)
xgboost_model_results = validate_xgboost_models(xgboost_data_df_tr, unlabel_psu_grps, grp_leads_to_keep_unlab)
xgboost_reports = fetch_xgboost_report(xgboost_data_df_tst, xgboost_trained_models, xgboost_model_results,  unlabel_psu_grps, grp_leads_to_keep_unlab)
xgboost_reports_df = pd.DataFrame(xgboost_reports)

C:\Users\gjm31\anaconda3\envs\rag_env\lib\site-packages\xgboost\core.py:158: UserWarning: [12:50:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\gjm31\anaconda3\envs\rag_env\lib\site-packages\xgboost\core.py:158: UserWarning: [12:50:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\gjm31\anaconda3\envs\rag_env\lib\site-packages\xgboost\core.py:158: UserWarning: [12:50:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\U

In [19]:
from tqdm import tqdm
results_dict = {'force_id': [], 'pt_report': [], 'xgb_report': [],  
                'psu_occlusion': [], 'pt_psu_gt': [], 'gpt_0shot_response': []
                
                # 'gpt_0shot_response': [], 'gpt_1shot_response': [], 'gpt_2shot_response': [], 'gpt_3shot_response': [], 'gpt_5shot_response': [], 'gpt_7shot_response': []
                }

# alt_reports_dict = {'force_id': [], 
#                        'close_cmr_fid1': [], 'close_pthx_fid1': [],
#                        'close_cmr_fid2': [], 'close_pthx_fid2': [],
#                        'close_cmr_fid3': [], 'close_pthx_fid3': [],
#                        'close_cmr_report1': [], 'close_pthx_report1': [],
#                        'close_cmr_report2': [], 'close_pthx_report2': [],
#                        'close_cmr_report3': [], 'close_pthx_report3': [],
#                        'close_cmr_psu_gt1': [], 'close_pthx_psu_gt1': [],
#                        'close_cmr_psu_gt2': [], 'close_pthx_psu_gt2': [],
#                        'close_cmr_psu_gt3': [], 'close_pthx_psu_gt3': []
#                        }

tst_fids = [fid for fid in tst_fids if 'BCH' == fid[0:3]]

# Initialize progress bars
outer_progress = tqdm(unlabel_psu_grps, desc="Outcomes", position=0)


for unlab_psu_group, psu_group in zip(outer_progress, label_psu_grps):
    psu_grp_lead = psu_group[0] # get name of variable that describes group of similar staus update outcome vars'
    psu_grp_lead_unlab = unlab_psu_group[0]
    if psu_grp_lead in grp_leads_to_keep:
        # Initialize inner loop progress bar
        inner_progress = tqdm(tst_fids, desc=f"FORCE IDs for outcome: ({psu_grp_lead})", position=1, leave=False)
        
        for fitr, fid in enumerate(inner_progress):
            
            main_fid_dict = get_fid_dict(fid, report_data_dict)
            fid_report_main, fid_psu_report_main = generate_patient_report(main_fid_dict, psu_occlusions=psu_group)
    
            cmr_sub_reports, cmr_psu_sub_reports = fetch_sub_reports('cmr', faiss_results['cmr'][fid][1],
                                                                     report_data_dict, psu_occlusions=psu_group)
            pthx_sub_reports, pthx_psu_sub_reports = fetch_sub_reports('pt_clinical_surgical_history', faiss_results['pt_clinical_surgical_history'][fid][0], 
                                                                       report_data_dict, psu_occlusions=psu_group)
            
            xgb_report = xgboost_reports_df[(xgboost_reports_df['force_id'] == fid) & (xgboost_reports_df['psu_occlusion'] == psu_grp_lead_unlab)]['xgboost_report'].iloc[0]
            # xgb_report = ''
            
            basic_prompt_resp = prompt_llm(fid_report_main, cmr_sub_reports, pthx_sub_reports, psu_grp_lead, top_k_matches=0, basic=True, xgb_report=xgb_report)
            # five_shot_prompt_resp = prompt_llm(fid_report_main, cmr_sub_reports, pthx_sub_reports, psu_grp_lead, top_k_matches=5, basic=False, xgb_report=xgb_report)
            # seven_shot_prompt_resp = prompt_llm(fid_report_main, cmr_sub_reports, pthx_sub_reports, psu_grp_lead, top_k_matches=7, basic=False, xgb_report=xgb_report)
            # fifteen_shot_prompt_resp = prompt_llm(fid_report_main, cmr_sub_reports, pthx_sub_reports, psu_grp_lead, top_k_matches=15, basic=False, xgb_report=xgb_report)
            
            # similar_cmr_fids = [dic['force_id'] for dic in faiss_results['cmr'][fid][1]]
            # similar_pthx_fids = [dic['force_id'] for dic in faiss_results['pt_clinical_surgical_history'][fid][0]]
            
            # alt_reports_dict['force_id'].append(fid)
            # alt_reports_dict['close_cmr_fid1'].append(similar_cmr_fids[0])
            # alt_reports_dict['close_pthx_fid1'].append(similar_pthx_fids[0])
            # alt_reports_dict['close_cmr_fid2'].append(similar_cmr_fids[1])
            # alt_reports_dict['close_pthx_fid2'].append(similar_pthx_fids[1])
            # alt_reports_dict['close_cmr_fid3'].append(similar_cmr_fids[2])
            # alt_reports_dict['close_pthx_fid3'].append(similar_pthx_fids[2])
            # alt_reports_dict['close_cmr_report1'].append(cmr_sub_reports[0])
            # alt_reports_dict['close_pthx_report1'].append(pthx_sub_reports[0])
            # alt_reports_dict['close_cmr_report2'].append(cmr_sub_reports[1])
            # alt_reports_dict['close_pthx_report2'].append(pthx_sub_reports[1])
            # alt_reports_dict['close_cmr_report3'].append(cmr_sub_reports[2])
            # alt_reports_dict['close_pthx_report3'].append(pthx_sub_reports[2])
            # alt_reports_dict['close_cmr_psu_gt1'].append(cmr_psu_sub_reports[0])
            # alt_reports_dict['close_pthx_psu_gt1'].append(pthx_psu_sub_reports[0])
            # alt_reports_dict['close_cmr_psu_gt2'].append(cmr_psu_sub_reports[1])
            # alt_reports_dict['close_pthx_psu_gt2'].append(pthx_psu_sub_reports[1])
            # alt_reports_dict['close_cmr_psu_gt3'].append(cmr_psu_sub_reports[2])
            # alt_reports_dict['close_pthx_psu_gt3'].append(pthx_psu_sub_reports[2])
            
            results_dict['force_id'].append(fid)
            results_dict['pt_report'].append(fid_report_main)
            results_dict['xgb_report'].append(xgb_report)
            if 'heart transplant' in psu_grp_lead.lower():
                results_dict['psu_occlusion'].append(psu_grp_lead.replace('Did the patient receive a heart transplant?', 'Did the patient receive or was listed for a heart transplant?'))
            else:
                results_dict['psu_occlusion'].append(psu_grp_lead)
            results_dict['pt_psu_gt'].append(fid_psu_report_main)            
            results_dict['gpt_0shot_response'].append(basic_prompt_resp)
            # results_dict['gpt_1shot_response'].append(one_shot_resp)
            # results_dict['gpt_2shot_response'].append(two_shot_prompt_resp)
            # results_dict['gpt_3shot_response'].append(three_shot_prompt_resp)
            # results_dict['gpt_5shot_response'].append(five_shot_prompt_resp)
            # results_dict['gpt_7shot_response'].append(seven_shot_prompt_resp)
            # results_dict['gpt_20shot_response'].append(fifteen_shot_prompt_resp)
        inner_progress.close()

outer_progress.close()
            

Outcomes:   0%|          | 0/21 [00:00<?, ?it/s]
FORCE IDs for outcome: (Does the patient have a diagnosis of seizures requiring antiepileptic medications?):   0%|          | 0/28 [00:00<?, ?it/s]
FORCE IDs for outcome: (Does the patient have a diagnosis of seizures requiring antiepileptic medications?):   4%|▎         | 1/28 [00:02<00:55,  2.06s/it]
FORCE IDs for outcome: (Does the patient have a diagnosis of seizures requiring antiepileptic medications?):   7%|▋         | 2/28 [00:04<00:55,  2.14s/it]
FORCE IDs for outcome: (Does the patient have a diagnosis of seizures requiring antiepileptic medications?):  11%|█         | 3/28 [00:08<01:21,  3.24s/it]
FORCE IDs for outcome: (Does the patient have a diagnosis of seizures requiring antiepileptic medications?):  14%|█▍        | 4/28 [00:10<01:06,  2.78s/it]
FORCE IDs for outcome: (Does the patient have a diagnosis of seizures requiring antiepileptic medications?):  18%|█▊        | 5/28 [00:15<01:23,  3.62s/it]
FORCE IDs for outcome: 

In [ ]:
print(pthx_psu_sub_reports[0])

In [21]:
results_df = pd.DataFrame(results_dict)
results_df['human_assessment0'] = ''
results_df['human_assessment1'] = ''
results_df['human_assessment2'] = ''
results_df['human_assessment3'] = ''
results_df['yesno_gt'] = ''

# alt_gt_results_df = pd.DataFrame(alt_reports_dict)
# results_df.sort_values('force_id').to_csv('results-BCHpts-5outcomes-zeroshot (2024-12-11).csv', index=False)
results_df.sort_values('force_id').to_csv('results-BCHpts-5outcomes-xgboost-zeroshot (2024-12-11).csv', index=False)
# alt_gt_results_df.sort_values('force_id').to_csv('alt-reports-BCHpts-7outcomes (2024-12-03).csv', index=False)

In [18]:
results_df

,force_id,pt_report,xgb_report,psu_occlusion,pt_psu_gt,gpt_0shot_response,human_assessment0,human_assessment1,human_assessment2,human_assessment3,yesno_gt
0,BCH-ALDNOR-1,Pt Status Event Update \n Has the patient die...,,Does the patient have a diagnosis of seizures ...,[ diagnosis of seizures requiring antiepilept...,"Based on the provided patient data, I would pr...",,,,,
1,BCH-BEABEN-1,Pt Status Event Update \n Has the patient die...,,Does the patient have a diagnosis of seizures ...,[ diagnosis of seizures requiring antiepilept...,"Based on the provided data, I would predict th...",,,,,
2,BCH-BYRSAM-1,Pt Status Event Update \n Has the patient die...,,Does the patient have a diagnosis of seizures ...,[ diagnosis of seizures requiring antiepilept...,"Based on the provided data, I would predict th...",,,,,
3,BCH-CASKEV-1,Pt Status Event Update \n Has the patient die...,,Does the patient have a diagnosis of seizures ...,[ diagnosis of seizures requiring antiepilept...,"Based on the provided data, the most likely pr...",,,,,
4,BCH-CIPEMI-1,Pt Status Event Update \n If other location o...,,Does the patient have a diagnosis of seizures ...,[ diagnosis of seizures requiring antiepilept...,"Based on the provided data, I would predict th...",,,,,
...,...,...,...,...,...,...,...,...,...,...,...
135,BCH-TARJOS-1,Pt Status Event Update \n If other location o...,,Did the patient receive or was listed for a he...,[ Prior listing for a heart transplant? no\n D...,"Based on the provided patient data, it is like...",,,,,
136,BCH-TOMCHR-1,Pt Status Event Update \n Has the patient die...,,Did the patient receive or was listed for a he...,[ Prior listing for a heart transplant? no\n D...,"Based on the provided data, it is reasonable t...",,,,,
137,BCH-TRATRI-1,Pt Status Event Update \n Has the patient die...,,Did the patient receive or was listed for a he...,[ Prior listing for a heart transplant? no\n D...,"Based on the provided data, I would predict th...",,,,,
138,BCH-VAZAGU-1,Pt Status Event Update \n Details of first em...,,Did the patient receive or was listed for a he...,[ Prior listing for a heart transplant? no\n D...,"Based on the provided information, I would pre...",,,,,
